In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]='1'

In [28]:
import boto3
from typing import List
from pathlib import Path
import polars as pl
import torch
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, MiniBatchKMeans, KMeans 
from transphorm.preprocessors import save_array_as_df_parquet
import scipy.fft as spft
import seaborn as sns
from transphorm.pipelines import reshape_for_multivariable, downsample
from sktime.clustering.k_means import TimeSeriesKMeans 
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [3]:
torch.device('mps')

device(type='mps')

In [52]:

main_path = Path("/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Gaby/Data Analysis/ActiveAvoidance/Core_guppy_postcross/core_data")
ts_path = main_path / "dopamine_full_timeseries_array.npy"
per_av_path = main_path/"percent_avoid.parquet"

data = torch.Tensor(np.load(ts_path))
data = data[~torch.isnan(data[:,0])]
# data = reshape_for_multivariable(data)

In [57]:
y = data[:,0]
X = data[:,1:]

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
# resahpe for multivariate with 1 channel- this allows for making any a model that is applicable to multivariate_
# def process_arr(arr):
#     return (downsample(
#             reshape_for_multivariable(arr)
#         ))

X_train, X_test = reshape_for_multivariable(X_train), reshape_for_multivariable(X_test) 


tensor([[[ 0.2768],
         [ 0.2723],
         [ 0.2679],
         ...,
         [-1.1605],
         [-1.1596],
         [-1.1587]],

        [[ 0.2170],
         [ 0.2282],
         [ 0.2395],
         ...,
         [ 1.3609],
         [ 1.3590],
         [ 1.3570]],

        [[ 2.6965],
         [ 2.6985],
         [ 2.7005],
         ...,
         [ 0.6903],
         [ 0.6837],
         [ 0.6770]],

        ...,

        [[-0.5125],
         [-0.5147],
         [-0.5169],
         ...,
         [-0.3195],
         [-0.3117],
         [-0.3038]],

        [[-0.3850],
         [-0.3753],
         [-0.3656],
         ...,
         [-0.6457],
         [-0.6432],
         [-0.6408]],

        [[-0.0443],
         [-0.0306],
         [-0.0170],
         ...,
         [ 0.3108],
         [ 0.3348],
         [ 0.3590]]])

In [ ]:
# get unique classes
classes = np.unique(y_train)
numb_classes = classes.shape[0]
numb_classes

In [15]:
km_classier = KNeighborsTimeSeriesClassifier(algorithm='brute_incr')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [16]:
classifier= KNeighborsTimeSeriesClassifier()
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

In [18]:
np.sum(predictions)

0.0

In [ ]:
from mpl_toolkits import mplot3d
min_threshold = np.min(np.max(pca, axis = 0))
min_threshold
pca_no_outliers = np.where(pca < min_threshold, pca, np.nan)
fig = plt.figure()
ax = plt.axes(projection = '3d')
pca_df = pl.DataFrame(pca_no_outliers, schema = ['p1', 'p2', 'p3']).with_columns(pl.lit(y).alias('learned'))
ax.scatter(pca_no_outliers[:,0], pca_no_outliers[:,2], pca_no_outliers[:,1], c = y)
# sns.scatterplot(pca_df, x = 'p2', y = 'p3', hue = 'learned', ax = ax)

In [ ]:
plot

In [ ]:
kmeans.labels_.shape
pca_df = pl.DataFrame(pca, schema = [f'pc_{i}' for i in range(pca.shape[1])]).with_columns(pl.lit(means.labels_).alias('clusters'), )
pca_df

In [ ]:
sample_rate = 1.0173e3
sec_1 = sample_rate
ms_200 = sec_1/5
ms_200

In [ ]:
example_class_1 = X_train[y_train==classes[0]][0]
example_class_2 = X_train[y_train==classes[1]][0]
example_class_2.shape

In [ ]:
plt.plot(example_class_1)
plt.plot(example_class_2)
plt.xscale('symlog')

In [ ]:
#shuffle data to use validation_split in training
idx = np.random.permutation(len(X_train))
X_train = X_train[idx]
y_train = y_train[idx]

In [ ]:
X_train.shape[-1]

In [ ]:
def transformer_encoder(inputs, headsize, num_heads, ff_dim, dropout = 0):
    # attention and normalization
    x = layers.MultiHeadAttention(
        key_dim = headsize,
        num_heads= num_heads, 
        dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x+inputs

    #feedforward
    x = layers.Conv1D(filters = ff_dim, kernel_size=1, activation='relu')(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters = inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

# build model
def build_model(
    input_shape, 
    head_size, 
    num_heads, 
    ff_dim,
    num_transformer_blocks, 
    mlp_units, 
    dropout = 0, 
    mlp_dropout = 0):
    
    inputs = keras.Input(shape = input_shape)
    x = inputs
    for x in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout = dropout)
    
    x = layers.GlobalAveragePooling1D(data_format = 'channels_last', dropout = dropout)(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation='relu')(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(numb_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs)

    
    
# train and eval
input_shape = X_train.shape[1:]
model = build_model(input_shape = input_shape, head_size=256, num_heads=4, ff_dim = 4, num_transformer_blocks=4, mlp_units = [128], mlp_dropout = 0.4, dropout=0.25)
model.compile(
    loss = "sparse_categorical_crossentropy", 
    optimizer = keras.optimizers.Adam(learning_rate = 1e-4),
    metrics = ['sparse_categorical_accuracy']
)
model.summary()

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=5, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(2, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 500
batch_size = 32
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only = True, monitor = "val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor = "val_loss", factor = 0.5, patience=0.5, min_lr = 0.0001
    ),
    keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 50, verbose = 1),
]
model.compile(
    optimizer = "adam", 
    loss = "sparse_categorical_crossentropy", 
    metrics = ["sparse_categorical_accuracy"],
)
history = model.fit(
    X_train,
    y_train,
    batch_size= batch_size, 
    epochs=epochs, 
    # callbacks=callbacks,
    validation_split=0.2, 
    verbose = 1
    )

In [ ]:

test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()